Severe Climate Events: Disaster Declaration for states and counties

In [402]:
import pandas as pd

In [403]:
disasters_raw = pd.read_csv("disaster_declarations.csv")


In [404]:
disasters_raw.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,hash,id,lastRefresh
0,FM-5444-TX,5444,TX,FM,2022-07-19T00:00:00.000Z,2022,Fire,CHALK MOUNTAIN FIRE,0,0,...,NaN,48,221,99221,Hood (County),22060,NaN,373c5ec27998afc08a53302dae796f476b1a6546,867be42a-71d5-4f13-aa21-d91e0a6fd577,2022-07-20T21:21:23.941Z
1,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,63,99063,Frontier (County),22034,NaN,ea3487ef36cff455236ce4c63d32fb8b5412bcef,e671348b-9782-42df-99f4-d38b8b1a89e6,2022-07-20T21:21:23.942Z
2,FM-5444-TX,5444,TX,FM,2022-07-19T00:00:00.000Z,2022,Fire,CHALK MOUNTAIN FIRE,0,0,...,NaN,48,425,99425,Somervell (County),22060,NaN,1f35dd8137e1b4cf003fb73d53d8aaaf642e6190,40f3ff75-0b80-4d25-8e85-156cd6a6f40b,2022-07-20T21:21:23.943Z
3,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,65,99065,Furnas (County),22034,NaN,9d813a845ab86f546bdf642fda53d3d4a0fbd098,de47838f-32db-4058-a34c-64997333789e,2022-07-20T21:21:23.943Z
4,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,145,99145,Red Willow (County),22034,NaN,d436856ce1d5205fa78597b93661fb27d3cea796,483c03b7-160a-414e-bb4f-5ccecc18d94a,2022-07-20T21:21:23.944Z


In [405]:
disasters_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63989 entries, 0 to 63988
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   femaDeclarationString     63989 non-null  object
 1   disasterNumber            63989 non-null  int64 
 2   state                     63989 non-null  object
 3   declarationType           63989 non-null  object
 4   declarationDate           63989 non-null  object
 5   fyDeclared                63989 non-null  int64 
 6   incidentType              63989 non-null  object
 7   declarationTitle          63989 non-null  object
 8   ihProgramDeclared         63989 non-null  int64 
 9   iaProgramDeclared         63989 non-null  int64 
 10  paProgramDeclared         63989 non-null  int64 
 11  hmProgramDeclared         63989 non-null  int64 
 12  incidentBeginDate         63989 non-null  object
 13  incidentEndDate           55609 non-null  object
 14  disasterCloseoutDate  

In [406]:
disasters = disasters_raw[['id','disasterNumber', 'state','fipsStateCode','fipsCountyCode','placeCode','declarationDate','fyDeclared','incidentType',
    'ihProgramDeclared','iaProgramDeclared','paProgramDeclared','hmProgramDeclared',
    'incidentBeginDate','incidentEndDate','designatedArea']]

Calculating the number of disasters by incident type and removing the ones that are not climate disasters. 

In [407]:
disasters.incidentType.value_counts()
disaster_type_list = disasters.incidentType.unique().tolist()
disaster_type_list
unwanted_disasters = ['Fire', 'Biological','Toxic Substances','Chemical','Terrorist','Human Cause','Fishing Losses']
climate_disasters_list = [ele for ele in disaster_type_list if ele not in unwanted_disasters]
climate_disasters = disasters.loc[disasters['incidentType'].isin(climate_disasters_list)]
climate_disasters

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea
36,036f0c80-8f7d-4551-a29b-92a7923f93a6,1606,TX,48,97,99097,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cooke (County)
37,d816361f-2c0e-431a-90e5-a836be2ffaf6,1606,TX,48,101,99101,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cottle (County)
57,a3c1ab84-73dd-4e1b-988f-9968c3a6f005,1971,AL,1,95,99095,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Marshall (County)
114,00c02cd1-ec76-47d1-a5e7-964c4fb6c7b5,4627,DE,10,3,99003,2021-10-24T00:00:00.000Z,2022,Hurricane,0,0,1,1,2021-09-01T00:00:00.000Z,2021-09-07T00:00:00.000Z,New Castle (County)
117,3fd860db-e27c-4211-9a69-b314bc68d306,1971,AL,1,119,99119,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Sumter (County)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63984,98e40c64-9ade-487f-9a5d-a61a05defd25,136,OR,41,0,0,1962-10-16T00:00:00.000Z,1963,Severe Storm,0,1,1,1,1962-10-16T00:00:00.000Z,1962-10-16T00:00:00.000Z,Statewide
63985,6d6279c0-0f40-4b39-9f5c-d1c7a3e83266,368,MS,28,157,99157,1973-03-27T00:00:00.000Z,1973,Severe Storm,0,1,1,0,1973-03-27T00:00:00.000Z,1973-03-27T00:00:00.000Z,Wilkinson (County)
63986,5dce9856-d591-471c-bfd2-21738c7f380a,330,MI,26,67,99067,1972-04-05T00:00:00.000Z,1972,Snowstorm,0,0,1,0,1972-04-05T00:00:00.000Z,1972-04-05T00:00:00.000Z,Ionia (County)
63987,33f53a3e-f779-4acd-b3d5-27862e400a28,297,OK,40,19,99019,1970-10-14T00:00:00.000Z,1971,Severe Storm,0,1,1,0,1970-10-14T00:00:00.000Z,1970-10-14T00:00:00.000Z,Carter (County)


In [408]:
climate_disasters.dtypes
climate_disasters["incidentBeginDate"] = pd.to_datetime(climate_disasters["incidentBeginDate"])
climate_disasters['Year'] = climate_disasters['incidentBeginDate'].dt.strftime('%Y')

/tmp/user/23091/ipykernel_1226135/252824159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_disasters["incidentBeginDate"] = pd.to_datetime(climate_disasters["incidentBeginDate"])
/tmp/user/23091/ipykernel_1226135/252824159.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_disasters['Year'] = climate_disasters['incidentBeginDate'].dt.strftime('%Y')


In [409]:
climate_disasters.head()

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea,Year
36,036f0c80-8f7d-4551-a29b-92a7923f93a6,1606,TX,48,97,99097,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23 00:00:00+00:00,2005-10-14T00:00:00.000Z,Cooke (County),2005
37,d816361f-2c0e-431a-90e5-a836be2ffaf6,1606,TX,48,101,99101,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23 00:00:00+00:00,2005-10-14T00:00:00.000Z,Cottle (County),2005
57,a3c1ab84-73dd-4e1b-988f-9968c3a6f005,1971,AL,1,95,99095,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15 00:00:00+00:00,2011-05-31T00:00:00.000Z,Marshall (County),2011
114,00c02cd1-ec76-47d1-a5e7-964c4fb6c7b5,4627,DE,10,3,99003,2021-10-24T00:00:00.000Z,2022,Hurricane,0,0,1,1,2021-09-01 00:00:00+00:00,2021-09-07T00:00:00.000Z,New Castle (County),2021
117,3fd860db-e27c-4211-9a69-b314bc68d306,1971,AL,1,119,99119,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15 00:00:00+00:00,2011-05-31T00:00:00.000Z,Sumter (County),2011


In [410]:
climate_last_23 = climate_disasters.loc[(climate_disasters['Year'] >= '2000')
                     & (climate_disasters['Year'] <= '2023')]
climate_last_23

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea,Year
36,036f0c80-8f7d-4551-a29b-92a7923f93a6,1606,TX,48,97,99097,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23 00:00:00+00:00,2005-10-14T00:00:00.000Z,Cooke (County),2005
37,d816361f-2c0e-431a-90e5-a836be2ffaf6,1606,TX,48,101,99101,2005-09-24T00:00:00.000Z,2005,Hurricane,0,0,1,1,2005-09-23 00:00:00+00:00,2005-10-14T00:00:00.000Z,Cottle (County),2005
57,a3c1ab84-73dd-4e1b-988f-9968c3a6f005,1971,AL,1,95,99095,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15 00:00:00+00:00,2011-05-31T00:00:00.000Z,Marshall (County),2011
114,00c02cd1-ec76-47d1-a5e7-964c4fb6c7b5,4627,DE,10,3,99003,2021-10-24T00:00:00.000Z,2022,Hurricane,0,0,1,1,2021-09-01 00:00:00+00:00,2021-09-07T00:00:00.000Z,New Castle (County),2021
117,3fd860db-e27c-4211-9a69-b314bc68d306,1971,AL,1,119,99119,2011-04-28T00:00:00.000Z,2011,Severe Storm,1,0,1,1,2011-04-15 00:00:00+00:00,2011-05-31T00:00:00.000Z,Sumter (County),2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58987,e5a603dd-b504-45f9-b30b-cfe7694f1055,1311,GA,13,223,99223,2000-01-28T00:00:00.000Z,2000,Severe Storm,0,0,1,1,2000-01-22 00:00:00+00:00,2000-02-01T00:00:00.000Z,Paulding (County),2000
59018,f607f5c6-1f6f-4ff7-a7f0-bda45598c22a,1311,GA,13,187,99187,2000-01-28T00:00:00.000Z,2000,Severe Storm,0,0,1,1,2000-01-22 00:00:00+00:00,2000-02-01T00:00:00.000Z,Lumpkin (County),2000
59019,5a26214d-7749-4d1f-968d-c7f00cd6b72e,1311,GA,13,311,99311,2000-01-28T00:00:00.000Z,2000,Severe Storm,0,0,1,1,2000-01-22 00:00:00+00:00,2000-02-01T00:00:00.000Z,White (County),2000
59053,a01f3466-8390-46ed-bc53-5304f1d9f54a,1311,GA,13,231,99231,2000-01-28T00:00:00.000Z,2000,Severe Storm,0,0,1,1,2000-01-22 00:00:00+00:00,2000-02-01T00:00:00.000Z,Pike (County),2000


If we take all years- we have 52,445 climate-related disasters. If we take from 2000 onwards (2000-2023 included), we get 33,629 climate related disasters. 

In [411]:
climate_last_23.state.value_counts()
#climate_last_23.designatedArea.value_counts()

TX    2363
MO    1845
LA    1832
OK    1603
KY    1509
FL    1404
GA    1272
VA    1227
NC    1219
KS    1175
MS    1134
PR    1072
AR     971
AL     971
IA     953
NE     921
NY     895
TN     790
IN     781
WV     713
SC     711
OH     680
ND     675
MN     667
SD     634
PA     600
IL     574
WI     384
CA     372
WA     351
NJ     343
MD     253
MA     240
MI     220
MT     214
ME     212
VT     197
OR     194
NH     193
CO     183
CT     171
NM     150
AK     111
ID     107
AZ      87
NV      75
UT      72
RI      65
HI      45
MP      38
VI      35
WY      32
AS      31
DE      30
DC      17
GU       8
FM       8
Name: state, dtype: int64

For political lens: get the name of the states that have been affected- maybe the highest ones?

For financial lens: counties that are yes and what all was provided to them? 

For socio-economic: counties that were affected- get the demographic breakdown, what are the relationships between the most heavily affected ones and their socio-economic breakdown?

In [412]:
#Renaming climate columns
climate_last_23.columns = ['record_id', 'disaster_number', 'state', 'state_code',
                          'county_code','place_code','dec_date', 'fiscal_year',
                        'disaster_type','ih_program','ia_program', 'pa_program',
                        'hm_progam','begin_date', 'end_date', 'area_name','year']
cliate_last_23 = climate_last_23.drop(columns=['fiscal_year'])
duplicateRows = climate_last_23[climate_last_23.duplicated()]
len(duplicateRows)
duplicateRows = climate_last_23[climate_last_23.duplicated(['record_id'])]
len(duplicateRows)

0

Does not look like there are any duplicates that we need to worry about. 
Hence, dropping the record_id column.

In [413]:
climate_last_23 = climate_last_23.drop(columns=['record_id'])


In [414]:
climate_last_23.hm_progam.value_counts()/33629

1    0.628505
0    0.371495
Name: hm_progam, dtype: float64

In [415]:
climate_last_23.ih_program.value_counts()/33629

0    0.80716
1    0.19284
Name: ih_program, dtype: float64

In [416]:
climate_last_23.ia_program.value_counts()/33629


0    0.86024
1    0.13976
Name: ia_program, dtype: float64

In [417]:
climate_last_23.pa_program.value_counts()/33629


1    0.941509
0    0.058491
Name: pa_program, dtype: float64

Hazard Mitigation declared: 21,136 disasters (62.85%)

Individual and Households: 27,144 disasters (80.71%)

Individual Assistance: 28,929 disasters (86.02%)

Public Assistance: 31,662 disasters. (94.15%)

Keeping only Public assistance column as decided by looking at other dataset.



In [418]:
climate_last_23 = climate_last_23.drop(columns=['ih_program','ia_program','hm_progam'])

Questions to answer:

1. What are the type of disasters that are occuring?

2. What % of disasters are getting some kind of economic assistance?

- Where does that economic assistance acctually flow to- which states?

3. Which months are causing what kind of disasters? Which month has the most disasters?

4. Which counties face the most disasters? 

5. Can add the public assistance data- map it to the specific disasters-
based on disaster numbers. See how much money goes to each state. Which kind of disaster gets the most amount of money as well. 
Use public assistance cause it is the largest grant that FEMA provides. 



Adding final_county_code by cleaning state code and county_code and merging them
together. 

In [420]:
climate_last_23["state_code"] = climate_last_23.state_code.map("{:02d}".format)

In [421]:
climate_last_23["county_code"] = climate_last_23.county_code.map("{:03d}".format)

In [422]:
climate_last_23['full_county_code'] = climate_last_23['state_code'] + climate_last_23['county_code']
climate_last_23 = climate_last_23.drop(columns=['place_code'])

What are the type of disasters and frequency of disasters? (Grouping by disasters and plotting them over time)

In [423]:
#How many disasters have we had?
climate_last_23.disaster_number.value_counts()
# Have had 1584 disasters 
#(climate_last_23['disaster_number'].value_counts() > 10)
#climate_last_23[["disaster_number"]].describe()
#climate_last_23['disaster_number_count'] = climate_last_23['disaster_number'].value_counts()


3261    255
1606    254
4586    254
3216    254
3554    254
       ... 
3300      1
1680      1
1654      1
3337      1
3581      1
Name: disaster_number, Length: 1584, dtype: int64

In [424]:
#climate_last_23.set_index(["disaster_type"], inplace = True, drop= False)
cl= climate_last_23.disaster_number.value_counts().to_dict()
climate_last_23['number_of_counties_affected'] = climate_last_23['disaster_number'].map(cl)

In [430]:
#climate_last_23.loc[1606]
climate_last_23['end_date']= pd.to_datetime(climate_last_23['end_date'])

climate_last_23['length_of_disaster'] = (climate_last_23['end_date'] - climate_last_23['begin_date']).dt.days
climate_last_23
climate_last_23.to_csv('climate_disasters_2000_2023_clean.csv')

Which disasters go on for the longest? 

In [431]:
climate_by_dis_num = climate_last_23.groupby(['disaster_type', 'disaster_number'])
climate_by_dis_num.first()

state state_code county_code  \
disaster_type     disaster_number                                
Coastal Storm     1379               TX         48         241   
                  1380               LA         22         011   
                  1393               FL         12         107   
                  1434               TX         48         167   
                  3223               UT         49         001   
...                                 ...        ...         ...   
Typhoon           4433               GU         66         010   
Volcanic Eruption 4201               HI         15         001   
                  4366               HI         15         001   
Winter Storm      3589               NY         36         009   
                  3590               NY         36         029   

                                                   dec_date  fiscal_year  \
disaster_type     disaster_number                                          
Coastal Storm     1379             2001-06-09T00:00:00.000Z         2001   
                  1380             2001-06-11T00:00:00.000Z         2001   
                  1393             2001-09-28T00:00:00.000Z         2001   
                  1434             2002-09-26T00:00:00.000Z         2002   
                  3223             2005-09-05T00:00:00.000Z         2005   
...                                                     ...          ...   
Typhoon           4433             2019-05-07T00:00:00.000Z         2019   
Volcanic Eruption 4201             2014-11-03T00:00:00.000Z         2015   
                  4366             2018-05-11T00:00:00.000Z         2018   
Winter Storm      3589             2022-11-20T00:00:00.000Z         2023   
                  3590             2022-12-26T00:00:00.000Z         2023   

                                   pa_program                begin_date  \
disaster_type     disaster_number                                         
Coastal Storm     1379                      1 2001-06-05 00:00:00+00:00   
                  1380                      1 2001-06-05 00:00:00+00:00   
                  1393                      1 2001-09-13 00:00:00+00:00   
                  1434                      0 2002-09-06 00:00:00+00:00   
                  3223                      1 2005-08-29 00:00:00+00:00   
...                                       ...                       ...   
Typhoon           4433                      1 2019-02-23 00:00:00+00:00   
Volcanic Eruption 4201                      1 2014-09-04 00:00:00+00:00   
                  4366                      1 2018-05-03 00:00:00+00:00   
Winter Storm      3589                      1 2022-11-18 00:00:00+00:00   
                  3590                      1 2022-12-23 00:00:00+00:00   

                                                   end_date  \
disaster_type     disaster_number                             
Coastal Storm     1379            2001-06-20 00:00:00+00:00   
                  1380            2001-06-22 00:00:00+00:00   
                  1393            2001-09-21 00:00:00+00:00   
                  1434            2002-09-30 00:00:00+00:00   
                  3223            2005-10-01 00:00:00+00:00   
...                                                     ...   
Typhoon           4433            2019-02-25 00:00:00+00:00   
Volcanic Eruption 4201            2015-03-25 00:00:00+00:00   
                  4366            2018-08-17 00:00:00+00:00   
Winter Storm      3589            2022-11-21 00:00:00+00:00   
                  3590            2022-12-28 00:00:00+00:00   

                                                  area_name  year  \
disaster_type     disaster_number                                   
Coastal Storm     1379                      Jasper (County)  2001   
                  1380                  Beauregard (Parish)  2001   
                  1393                      Putnam (County)  2001   
                  1434                   Galv

Total number of disasters each year?

In [432]:
climate_last_23.groupby('disaster_number')['state_code'].unique().count()

1584

In [433]:
climate_last_23.loc[climate_last_23["disaster_number"]==1311]

,disaster_number,state,state_code,county_code,dec_date,fiscal_year,disaster_type,pa_program,begin_date,end_date,area_name,year,full_county_code,number_of_counties_affected,length_of_disaster
58626,1311,GA,13,143,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Haralson (County),2000,13143,51,10.0
58656,1311,GA,13,035,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Butts (County),2000,13035,51,10.0
58657,1311,GA,13,057,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Cherokee (County),2000,13057,51,10.0
58658,1311,GA,13,117,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Forsyth (County),2000,13117,51,10.0
58659,1311,GA,13,151,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Henry (County),2000,13151,51,10.0
58660,1311,GA,13,297,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Walton (County),2000,13297,51,10.0
58724,1311,GA,13,011,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Banks (County),2000,13011,51,10.0
58725,1311,GA,13,097,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Douglas (County),2000,13097,51,10.0
58726,1311,GA,13,133,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Greene (County),2000,13133,51,10.0
58727,1311,GA,13,139,2000-01-28T00:00:00.000Z,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Hall (County),2000,13139,51,10.0


In [434]:
#climate_last_23.full_county_code.value_counts()
climate_dict = {k: f.groupby('disaster_number')['year'].unique().apply(list).to_dict()
     for k, f in climate_last_23.groupby('disaster_type')}

d = {}
for key, value in climate_dict.items():
    d[key] = {}
    d[key][set(value)] 
climate_dict['Coastal Storm'].values()
#climate_dict = climate_last_23.groupby('disaster_type').apply(lambda x: x.set_index(['disaster_number','full_county_code']).to_dict(orient='index')).to_dict()

TypeError: unhashable type: 'set'